# Advanced Query Management

## Learning Objectives
By the end of this notebook, you will be able to:
- Write complex SQL queries with joins, subqueries, and aggregations
- Optimize query performance using indexes and query planning
- Handle transactions and maintain data consistency
- Use parameterized queries for security and performance
- Implement data analysis queries for business intelligence

## 1. Complex Query Patterns

In [ ]:
import sqlite3
import pandas as pd
from typing import List, Dict, Any, Optional, Tuple
from contextlib import contextmanager
import numpy as np
from datetime import datetime, timedelta

@contextmanager
def get_db_connection(db_path: str = "advanced_company.db"):
    """
    Context manager for database connections.
    """
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row
    conn.execute("PRAGMA foreign_keys = ON")
    try:
        yield conn
    finally:
        conn.close()

# Create comprehensive database with sample data
def setup_advanced_database() -> bool:
    """
    Set up a comprehensive database for advanced query examples.
    
    Returns:
        True if successful, False otherwise
    """
    
    try:
        with get_db_connection() as conn:
            # Create tables
            conn.executescript("""
            -- Departments table
            CREATE TABLE IF NOT EXISTS departments (
                department_id INTEGER PRIMARY KEY,
                department_name VARCHAR(100) NOT NULL,
                manager_id INTEGER,
                budget DECIMAL(15, 2),
                location VARCHAR(100)
            );
            
            -- Employees table
            CREATE TABLE IF NOT EXISTS employees (
                employee_id INTEGER PRIMARY KEY,
                first_name VARCHAR(50) NOT NULL,
                last_name VARCHAR(50) NOT NULL,
                email VARCHAR(100) UNIQUE,
                hire_date DATE NOT NULL,
                department_id INTEGER,
                position VARCHAR(100),
                salary DECIMAL(10, 2),
                manager_id INTEGER,
                is_active BOOLEAN DEFAULT 1,
                FOREIGN KEY (department_id) REFERENCES departments(department_id),
                FOREIGN KEY (manager_id) REFERENCES employees(employee_id)
            );
            
            -- Projects table
            CREATE TABLE IF NOT EXISTS projects (
                project_id INTEGER PRIMARY KEY,
                project_name VARCHAR(200) NOT NULL,
                start_date DATE,
                end_date DATE,
                budget DECIMAL(15, 2),
                status VARCHAR(20) DEFAULT 'ACTIVE',
                department_id INTEGER,
                FOREIGN KEY (department_id) REFERENCES departments(department_id)
            );
            
            -- Employee project assignments
            CREATE TABLE IF NOT EXISTS project_assignments (
                assignment_id INTEGER PRIMARY KEY,
                employee_id INTEGER,
                project_id INTEGER,
                role VARCHAR(50),
                hours_allocated DECIMAL(5, 2),
                start_date DATE,
                end_date DATE,
                FOREIGN KEY (employee_id) REFERENCES employees(employee_id),
                FOREIGN KEY (project_id) REFERENCES projects(project_id)
            );
            
            -- Time tracking
            CREATE TABLE IF NOT EXISTS time_entries (
                entry_id INTEGER PRIMARY KEY,
                employee_id INTEGER,
                project_id INTEGER,
                work_date DATE,
                hours_worked DECIMAL(4, 2),
                description TEXT,
                FOREIGN KEY (employee_id) REFERENCES employees(employee_id),
                FOREIGN KEY (project_id) REFERENCES projects(project_id)
            );
            """)
            
            # Insert sample data
            # Departments
            departments_data = [
                (1, 'Engineering', None, 500000, 'Building A'),
                (2, 'Marketing', None, 200000, 'Building B'),
                (3, 'Sales', None, 300000, 'Building B'),
                (4, 'HR', None, 150000, 'Building C'),
                (5, 'Finance', None, 250000, 'Building C')
            ]
            
            conn.executemany(
                "INSERT OR REPLACE INTO departments VALUES (?, ?, ?, ?, ?)",
                departments_data
            )
            
            # Employees
            employees_data = [
                (1, 'Alice', 'Johnson', 'alice@company.com', '2020-01-15', 1, 'Engineering Manager', 95000, None, 1),
                (2, 'Bob', 'Smith', 'bob@company.com', '2021-03-01', 1, 'Senior Developer', 85000, 1, 1),
                (3, 'Charlie', 'Brown', 'charlie@company.com', '2021-06-15', 1, 'Developer', 75000, 1, 1),
                (4, 'Diana', 'Davis', 'diana@company.com', '2019-11-20', 2, 'Marketing Manager', 80000, None, 1),
                (5, 'Eve', 'Wilson', 'eve@company.com', '2022-02-01', 2, 'Marketing Specialist', 60000, 4, 1),
                (6, 'Frank', 'Miller', 'frank@company.com', '2020-08-10', 3, 'Sales Manager', 90000, None, 1),
                (7, 'Grace', 'Lee', 'grace@company.com', '2021-12-01', 3, 'Sales Rep', 65000, 6, 1),
                (8, 'Henry', 'Clark', 'henry@company.com', '2022-01-15', 4, 'HR Specialist', 55000, None, 1),
                (9, 'Ivy', 'Taylor', 'ivy@company.com', '2020-05-01', 5, 'Financial Analyst', 70000, None, 1),
                (10, 'Jack', 'White', 'jack@company.com', '2023-01-01', 1, 'Junior Developer', 65000, 1, 1)
            ]
            
            conn.executemany(
                "INSERT OR REPLACE INTO employees VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
                employees_data
            )
            
            # Projects
            projects_data = [
                (1, 'Customer Portal Redesign', '2024-01-01', '2024-06-30', 150000, 'ACTIVE', 1),
                (2, 'Marketing Campaign Q1', '2024-01-01', '2024-03-31', 75000, 'COMPLETED', 2),
                (3, 'Sales Process Automation', '2024-02-01', '2024-08-31', 200000, 'ACTIVE', 3),
                (4, 'HR System Upgrade', '2024-03-01', '2024-09-30', 100000, 'PLANNING', 4),
                (5, 'Financial Reporting Dashboard', '2024-01-15', '2024-07-15', 120000, 'ACTIVE', 5)
            ]
            
            conn.executemany(
                "INSERT OR REPLACE INTO projects VALUES (?, ?, ?, ?, ?, ?, ?)",
                projects_data
            )
            
            # Project assignments
            assignments_data = [
                (1, 1, 1, 'Project Manager', 40, '2024-01-01', '2024-06-30'),
                (2, 2, 1, 'Lead Developer', 40, '2024-01-01', '2024-06-30'),
                (3, 3, 1, 'Developer', 40, '2024-01-15', '2024-06-30'),
                (4, 10, 1, 'Junior Developer', 30, '2024-02-01', '2024-06-30'),
                (5, 4, 2, 'Campaign Manager', 40, '2024-01-01', '2024-03-31'),
                (6, 5, 2, 'Content Creator', 35, '2024-01-01', '2024-03-31'),
                (7, 6, 3, 'Sales Lead', 30, '2024-02-01', '2024-08-31'),
                (8, 7, 3, 'Sales Analyst', 25, '2024-02-15', '2024-08-31'),
                (9, 9, 5, 'Financial Analyst', 35, '2024-01-15', '2024-07-15')
            ]
            
            conn.executemany(
                "INSERT OR REPLACE INTO project_assignments VALUES (?, ?, ?, ?, ?, ?, ?)",
                assignments_data
            )
            
            # Time entries (sample data for recent weeks)
            time_entries_data = []
            base_date = datetime(2024, 3, 1)
            
            # Generate time entries for active projects
            for day_offset in range(20):  # 20 working days
                current_date = base_date + timedelta(days=day_offset)
                if current_date.weekday() < 5:  # Weekdays only
                    date_str = current_date.strftime('%Y-%m-%d')
                    
                    # Engineering team on Portal project
                    time_entries_data.extend([
                        (None, 1, 1, date_str, 8.0, 'Project management and planning'),
                        (None, 2, 1, date_str, 7.5, 'Backend development'),
                        (None, 3, 1, date_str, 8.0, 'Frontend development'),
                        (None, 10, 1, date_str, 6.0, 'Testing and bug fixes')
                    ])
                    
                    # Finance team on Dashboard project
                    time_entries_data.append(
                        (None, 9, 5, date_str, 7.0, 'Requirements analysis and testing')
                    )
            
            conn.executemany(
                "INSERT INTO time_entries (employee_id, project_id, work_date, hours_worked, description) VALUES (?, ?, ?, ?, ?)",
                time_entries_data
            )
            
            conn.commit()
            print("✓ Advanced database setup completed successfully")
            return True
            
    except sqlite3.Error as e:
        print(f"Error setting up database: {e}")
        return False

# Set up the database
setup_advanced_database()

In [ ]:
# Complex JOIN queries
def demonstrate_complex_joins() -> None:
    """
    Demonstrate various types of complex JOIN operations.
    """
    
    try:
        with get_db_connection() as conn:
            
            print("=== Complex JOIN Queries ===")
            
            # 1. Multi-table JOIN with employee hierarchy
            print("\n1. Employee Hierarchy with Department Information:")
            hierarchy_query = """
            SELECT 
                e.first_name || ' ' || e.last_name as employee_name,
                e.position,
                d.department_name,
                m.first_name || ' ' || m.last_name as manager_name,
                e.salary
            FROM employees e
            JOIN departments d ON e.department_id = d.department_id
            LEFT JOIN employees m ON e.manager_id = m.employee_id
            WHERE e.is_active = 1
            ORDER BY d.department_name, e.salary DESC
            """
            
            cursor = conn.execute(hierarchy_query)
            results = cursor.fetchall()
            
            for row in results:
                manager = row['manager_name'] if row['manager_name'] else 'No Manager'
                print(f"  {row['employee_name']} ({row['department_name']}) - {row['position']}")
                print(f"    Manager: {manager}, Salary: ${row['salary']:,.2f}")
            
            # 2. Project assignments with employee and project details
            print("\n2. Active Project Assignments:")
            assignments_query = """
            SELECT 
                p.project_name,
                p.status,
                e.first_name || ' ' || e.last_name as employee_name,
                pa.role,
                pa.hours_allocated,
                d.department_name
            FROM project_assignments pa
            JOIN employees e ON pa.employee_id = e.employee_id
            JOIN projects p ON pa.project_id = p.project_id
            JOIN departments d ON e.department_id = d.department_id
            WHERE p.status = 'ACTIVE'
            ORDER BY p.project_name, pa.hours_allocated DESC
            """
            
            cursor = conn.execute(assignments_query)
            results = cursor.fetchall()
            
            current_project = None
            for row in results:
                if row['project_name'] != current_project:
                    current_project = row['project_name']
                    print(f"\n  📋 {current_project} ({row['status']})")
                
                print(f"    • {row['employee_name']} ({row['department_name']}) - {row['role']} ({row['hours_allocated']}h/week)")
            
            # 3. Cross-department collaboration analysis
            print("\n3. Cross-Department Project Collaboration:")
            collaboration_query = """
            SELECT 
                p.project_name,
                COUNT(DISTINCT e.department_id) as departments_involved,
                GROUP_CONCAT(DISTINCT d.department_name) as departments,
                COUNT(pa.employee_id) as total_employees,
                SUM(pa.hours_allocated) as total_hours_allocated
            FROM projects p
            JOIN project_assignments pa ON p.project_id = pa.project_id
            JOIN employees e ON pa.employee_id = e.employee_id
            JOIN departments d ON e.department_id = d.department_id
            WHERE p.status IN ('ACTIVE', 'PLANNING')
            GROUP BY p.project_id, p.project_name
            HAVING COUNT(DISTINCT e.department_id) > 1
            ORDER BY departments_involved DESC, total_hours_allocated DESC
            """
            
            cursor = conn.execute(collaboration_query)
            results = cursor.fetchall()
            
            for row in results:
                print(f"  {row['project_name']}:")
                print(f"    Departments: {row['departments']} ({row['departments_involved']} depts)")
                print(f"    Resources: {row['total_employees']} employees, {row['total_hours_allocated']} hours/week")
                
    except sqlite3.Error as e:
        print(f"Error in JOIN queries: {e}")

demonstrate_complex_joins()

In [ ]:
# Subqueries and CTEs (Common Table Expressions)
def demonstrate_subqueries_and_ctes() -> None:
    """
    Demonstrate subqueries and Common Table Expressions.
    """
    
    try:
        with get_db_connection() as conn:
            
            print("=== Subqueries and CTEs ===")
            
            # 1. Subquery to find employees earning above department average
            print("\n1. Employees Earning Above Department Average:")
            subquery_example = """
            SELECT 
                e.first_name || ' ' || e.last_name as employee_name,
                d.department_name,
                e.salary,
                dept_avg.avg_salary,
                ROUND((e.salary - dept_avg.avg_salary) / dept_avg.avg_salary * 100, 2) as percent_above_avg
            FROM employees e
            JOIN departments d ON e.department_id = d.department_id
            JOIN (
                SELECT 
                    department_id,
                    AVG(salary) as avg_salary
                FROM employees 
                WHERE is_active = 1
                GROUP BY department_id
            ) dept_avg ON e.department_id = dept_avg.department_id
            WHERE e.salary > dept_avg.avg_salary
            ORDER BY percent_above_avg DESC
            """
            
            cursor = conn.execute(subquery_example)
            results = cursor.fetchall()
            
            for row in results:
                print(f"  {row['employee_name']} ({row['department_name']})")
                print(f"    Salary: ${row['salary']:,.2f} vs Dept Avg: ${row['avg_salary']:,.2f} (+{row['percent_above_avg']}%)")
            
            # 2. CTE for recursive employee hierarchy
            print("\n2. Employee Hierarchy using CTE:")
            cte_hierarchy = """
            WITH RECURSIVE employee_hierarchy AS (
                -- Base case: top-level managers (no manager)
                SELECT 
                    employee_id,
                    first_name || ' ' || last_name as employee_name,
                    position,
                    manager_id,
                    0 as level,
                    first_name || ' ' || last_name as hierarchy_path
                FROM employees 
                WHERE manager_id IS NULL AND is_active = 1
                
                UNION ALL
                
                -- Recursive case: employees with managers
                SELECT 
                    e.employee_id,
                    e.first_name || ' ' || e.last_name as employee_name,
                    e.position,
                    e.manager_id,
                    eh.level + 1,
                    eh.hierarchy_path || ' -> ' || e.first_name || ' ' || e.last_name
                FROM employees e
                JOIN employee_hierarchy eh ON e.manager_id = eh.employee_id
                WHERE e.is_active = 1
            )
            SELECT 
                employee_name,
                position,
                level,
                hierarchy_path
            FROM employee_hierarchy
            ORDER BY level, employee_name
            """
            
            cursor = conn.execute(cte_hierarchy)
            results = cursor.fetchall()
            
            for row in results:
                indent = "  " * row['level']
                print(f"{indent}Level {row['level']}: {row['employee_name']} - {row['position']}")
                if row['level'] > 0:
                    print(f"{indent}  Path: {row['hierarchy_path']}")
            
            # 3. Complex CTE for project resource analysis
            print("\n3. Project Resource Analysis using CTE:")
            resource_analysis = """
            WITH project_stats AS (
                SELECT 
                    p.project_id,
                    p.project_name,
                    p.budget,
                    p.status,
                    COUNT(pa.employee_id) as team_size,
                    SUM(pa.hours_allocated) as total_hours_allocated,
                    AVG(e.salary) as avg_team_salary
                FROM projects p
                LEFT JOIN project_assignments pa ON p.project_id = pa.project_id
                LEFT JOIN employees e ON pa.employee_id = e.employee_id
                GROUP BY p.project_id, p.project_name, p.budget, p.status
            ),
            time_tracking_stats AS (
                SELECT 
                    project_id,
                    COUNT(*) as total_time_entries,
                    SUM(hours_worked) as total_hours_logged,
                    AVG(hours_worked) as avg_daily_hours
                FROM time_entries
                GROUP BY project_id
            )
            SELECT 
                ps.project_name,
                ps.status,
                ps.budget,
                ps.team_size,
                ps.total_hours_allocated,
                COALESCE(tts.total_hours_logged, 0) as hours_logged,
                ROUND(ps.avg_team_salary, 2) as avg_team_salary,
                ROUND(ps.budget / NULLIF(ps.team_size, 0), 2) as budget_per_person
            FROM project_stats ps
            LEFT JOIN time_tracking_stats tts ON ps.project_id = tts.project_id
            WHERE ps.team_size > 0
            ORDER BY ps.budget DESC
            """
            
            cursor = conn.execute(resource_analysis)
            results = cursor.fetchall()
            
            for row in results:
                print(f"  📊 {row['project_name']} ({row['status']})")
                print(f"    Budget: ${row['budget']:,.2f} ({row['team_size']} people = ${row['budget_per_person']:,.2f}/person)")
                print(f"    Resources: {row['total_hours_allocated']}h allocated, {row['hours_logged']}h logged")
                print(f"    Team avg salary: ${row['avg_team_salary']:,.2f}")
                
    except sqlite3.Error as e:
        print(f"Error in subqueries/CTEs: {e}")

demonstrate_subqueries_and_ctes()

## 2. Aggregation and Window Functions

In [ ]:
# Advanced aggregation queries
def demonstrate_advanced_aggregations() -> None:
    """
    Demonstrate advanced aggregation and analytical queries.
    """
    
    try:
        with get_db_connection() as conn:
            
            print("=== Advanced Aggregations ===")
            
            # 1. Department statistics with multiple aggregations
            print("\n1. Comprehensive Department Statistics:")
            dept_stats_query = """
            SELECT 
                d.department_name,
                d.location,
                COUNT(e.employee_id) as employee_count,
                ROUND(AVG(e.salary), 2) as avg_salary,
                MIN(e.salary) as min_salary,
                MAX(e.salary) as max_salary,
                ROUND(AVG(e.salary) * COUNT(e.employee_id), 2) as total_payroll,
                d.budget,
                ROUND((d.budget - (AVG(e.salary) * COUNT(e.employee_id))) / d.budget * 100, 2) as budget_utilization_pct
            FROM departments d
            LEFT JOIN employees e ON d.department_id = e.department_id AND e.is_active = 1
            GROUP BY d.department_id, d.department_name, d.location, d.budget
            HAVING employee_count > 0
            ORDER BY total_payroll DESC
            """
            
            cursor = conn.execute(dept_stats_query)
            results = cursor.fetchall()
            
            for row in results:
                print(f"  🏢 {row['department_name']} ({row['location']})")
                print(f"    Team: {row['employee_count']} employees")
                print(f"    Salary Range: ${row['min_salary']:,.2f} - ${row['max_salary']:,.2f} (avg: ${row['avg_salary']:,.2f})")
                print(f"    Budget: ${row['budget']:,.2f}, Payroll: ${row['total_payroll']:,.2f} ({row['budget_utilization_pct']}% remaining)")
            
            # 2. Time tracking analysis with window functions
            print("\n2. Employee Productivity Analysis (Window Functions):")
            productivity_query = """
            SELECT 
                e.first_name || ' ' || e.last_name as employee_name,
                d.department_name,
                COUNT(te.entry_id) as total_entries,
                ROUND(SUM(te.hours_worked), 2) as total_hours,
                ROUND(AVG(te.hours_worked), 2) as avg_daily_hours,
                ROUND(SUM(te.hours_worked) / COUNT(DISTINCT te.work_date), 2) as avg_hours_per_day,
                ROW_NUMBER() OVER (ORDER BY SUM(te.hours_worked) DESC) as productivity_rank,
                ROUND(SUM(te.hours_worked) * 100.0 / SUM(SUM(te.hours_worked)) OVER (), 2) as percent_of_total_hours,
                ROUND(AVG(SUM(te.hours_worked)) OVER (PARTITION BY d.department_id), 2) as dept_avg_hours
            FROM employees e
            JOIN departments d ON e.department_id = d.department_id
            JOIN time_entries te ON e.employee_id = te.employee_id
            WHERE e.is_active = 1
            GROUP BY e.employee_id, e.first_name, e.last_name, d.department_name, d.department_id
            ORDER BY total_hours DESC
            """
            
            cursor = conn.execute(productivity_query)
            results = cursor.fetchall()
            
            for row in results:
                print(f"  #{row['productivity_rank']} {row['employee_name']} ({row['department_name']})")
                print(f"    Hours: {row['total_hours']} total ({row['avg_hours_per_day']}/day avg) - {row['percent_of_total_hours']}% of company total")
                print(f"    vs Dept Avg: {row['dept_avg_hours']} hours")
            
            # 3. Project timeline and milestone analysis
            print("\n3. Project Timeline Analysis:")
            timeline_query = """
            SELECT 
                p.project_name,
                p.status,
                p.start_date,
                p.end_date,
                JULIANDAY(p.end_date) - JULIANDAY(p.start_date) as planned_duration_days,
                JULIANDAY('now') - JULIANDAY(p.start_date) as days_elapsed,
                CASE 
                    WHEN p.status = 'COMPLETED' THEN 100
                    WHEN p.status = 'ACTIVE' THEN 
                        ROUND((JULIANDAY('now') - JULIANDAY(p.start_date)) / 
                              (JULIANDAY(p.end_date) - JULIANDAY(p.start_date)) * 100, 2)
                    ELSE 0
                END as completion_percentage,
                COUNT(pa.employee_id) as team_size,
                COALESCE(SUM(te.hours_worked), 0) as total_hours_logged
            FROM projects p
            LEFT JOIN project_assignments pa ON p.project_id = pa.project_id
            LEFT JOIN time_entries te ON p.project_id = te.project_id
            GROUP BY p.project_id, p.project_name, p.status, p.start_date, p.end_date
            ORDER BY 
                CASE p.status 
                    WHEN 'ACTIVE' THEN 1 
                    WHEN 'PLANNING' THEN 2 
                    WHEN 'COMPLETED' THEN 3 
                END,
                completion_percentage DESC
            """
            
            cursor = conn.execute(timeline_query)
            results = cursor.fetchall()
            
            for row in results:
                status_emoji = {'ACTIVE': '🟢', 'PLANNING': '🟡', 'COMPLETED': '✅'}.get(row['status'], '⚪')
                print(f"  {status_emoji} {row['project_name']} ({row['status']})")
                print(f"    Timeline: {row['start_date']} to {row['end_date']} ({row['planned_duration_days']:.0f} days)")
                print(f"    Progress: {row['completion_percentage']:.1f}% complete ({row['days_elapsed']:.0f} days elapsed)")
                print(f"    Resources: {row['team_size']} team members, {row['total_hours_logged']} hours logged")
                
    except sqlite3.Error as e:
        print(f"Error in aggregation queries: {e}")

demonstrate_advanced_aggregations()

## 3. Query Optimization and Performance

In [ ]:
# Query optimization techniques
def demonstrate_query_optimization() -> None:
    """
    Demonstrate query optimization techniques and performance analysis.
    """
    
    try:
        with get_db_connection() as conn:
            
            print("=== Query Optimization ===")
            
            # 1. Create indexes for better performance
            print("\n1. Creating Performance Indexes:")
            index_statements = [
                "CREATE INDEX IF NOT EXISTS idx_employees_department ON employees(department_id)",
                "CREATE INDEX IF NOT EXISTS idx_employees_manager ON employees(manager_id)",
                "CREATE INDEX IF NOT EXISTS idx_employees_salary ON employees(salary)",
                "CREATE INDEX IF NOT EXISTS idx_project_assignments_employee ON project_assignments(employee_id)",
                "CREATE INDEX IF NOT EXISTS idx_project_assignments_project ON project_assignments(project_id)",
                "CREATE INDEX IF NOT EXISTS idx_time_entries_employee_date ON time_entries(employee_id, work_date)",
                "CREATE INDEX IF NOT EXISTS idx_time_entries_project_date ON time_entries(project_id, work_date)"
            ]
            
            for index_sql in index_statements:
                conn.execute(index_sql)
                index_name = index_sql.split()[-3]  # Extract index name
                print(f"  ✓ Created index: {index_name}")
            
            # 2. Query execution plan analysis
            print("\n2. Query Execution Plan Analysis:")
            
            # Example query to analyze
            sample_query = """
            SELECT 
                e.first_name || ' ' || e.last_name as employee_name,
                d.department_name,
                SUM(te.hours_worked) as total_hours
            FROM employees e
            JOIN departments d ON e.department_id = d.department_id
            JOIN time_entries te ON e.employee_id = te.employee_id
            WHERE te.work_date >= '2024-03-01'
            GROUP BY e.employee_id, e.first_name, e.last_name, d.department_name
            ORDER BY total_hours DESC
            """
            
            # Get query plan
            cursor = conn.execute(f"EXPLAIN QUERY PLAN {sample_query}")
            plan_results = cursor.fetchall()
            
            print(f"  Query execution plan:")
            for row in plan_results:
                print(f"    {row[0]}: {row[3]}")
            
            # 3. Performance comparison: with and without indexes
            print("\n3. Performance Optimization Examples:")
            
            # Optimized query using indexes
            optimized_query = """
            SELECT 
                d.department_name,
                COUNT(e.employee_id) as employee_count,
                AVG(e.salary) as avg_salary
            FROM departments d
            JOIN employees e ON d.department_id = e.department_id
            WHERE e.is_active = 1 AND e.salary > 60000
            GROUP BY d.department_id, d.department_name
            ORDER BY avg_salary DESC
            """
            
            cursor = conn.execute(optimized_query)
            results = cursor.fetchall()
            
            print(f"  Optimized query results:")
            for row in results:
                print(f"    {row['department_name']}: {row['employee_count']} employees, avg salary: ${row['avg_salary']:,.2f}")
            
            # 4. Query optimization best practices
            print("\n4. Query Optimization Best Practices:")
            
            best_practices = [
                "✓ Use indexes on frequently queried columns (department_id, employee_id)",
                "✓ Use LIMIT when you don't need all results",
                "✓ Use specific column names instead of SELECT *",
                "✓ Use WHERE clauses to filter data early",
                "✓ Use appropriate JOIN types (INNER vs LEFT JOIN)",
                "✓ Use EXPLAIN QUERY PLAN to analyze performance",
                "✓ Consider denormalization for frequently accessed data",
                "✓ Use parameterized queries for security and performance"
            ]
            
            for practice in best_practices:
                print(f"    {practice}")
            
            # 5. Parameterized query example
            print("\n5. Parameterized Query Example:")
            
            def get_employees_by_department(dept_name: str, min_salary: float) -> List[sqlite3.Row]:
                """
                Get employees by department with minimum salary using parameterized query.
                """
                parameterized_query = """
                SELECT 
                    e.first_name || ' ' || e.last_name as employee_name,
                    e.position,
                    e.salary,
                    e.hire_date
                FROM employees e
                JOIN departments d ON e.department_id = d.department_id
                WHERE d.department_name = ? AND e.salary >= ? AND e.is_active = 1
                ORDER BY e.salary DESC
                """
                
                cursor = conn.execute(parameterized_query, (dept_name, min_salary))
                return cursor.fetchall()
            
            # Test parameterized query
            engineering_employees = get_employees_by_department("Engineering", 70000)
            
            print(f"  Engineering employees with salary >= $70,000:")
            for emp in engineering_employees:
                print(f"    {emp['employee_name']} - {emp['position']} (${emp['salary']:,.2f}, hired {emp['hire_date']})")
                
    except sqlite3.Error as e:
        print(f"Error in query optimization: {e}")

demonstrate_query_optimization()

## 4. Transaction Management

In [ ]:
# Transaction management and data consistency
def demonstrate_transaction_management() -> None:
    """
    Demonstrate transaction management for data consistency.
    """
    
    print("=== Transaction Management ===")
    
    # 1. Basic transaction example
    print("\n1. Basic Transaction Example:")
    
    def transfer_employee_to_department(employee_id: int, new_dept_id: int, new_salary: float) -> bool:
        """
        Transfer employee to new department with salary update in a transaction.
        """
        try:
            with get_db_connection() as conn:
                # Start transaction (implicit with context manager)
                
                # Get current employee info
                cursor = conn.execute(
                    "SELECT first_name, last_name, department_id, salary FROM employees WHERE employee_id = ?",
                    (employee_id,)
                )
                current_info = cursor.fetchone()
                
                if not current_info:
                    print(f"    Employee {employee_id} not found")
                    return False
                
                print(f"    Transferring {current_info['first_name']} {current_info['last_name']}")
                print(f"    From dept {current_info['department_id']} to dept {new_dept_id}")
                print(f"    Salary change: ${current_info['salary']:,.2f} -> ${new_salary:,.2f}")
                
                # Update employee department and salary
                conn.execute(
                    "UPDATE employees SET department_id = ?, salary = ?, updated_at = CURRENT_TIMESTAMP WHERE employee_id = ?",
                    (new_dept_id, new_salary, employee_id)
                )
                
                # Log the change in salary history (if table exists)
                try:
                    conn.execute(
                        "INSERT INTO salary_history (employee_id, old_salary, new_salary, change_date, change_reason) VALUES (?, ?, ?, date('now'), ?)",
                        (employee_id, current_info['salary'], new_salary, f"Department transfer to dept {new_dept_id}")
                    )
                except sqlite3.Error:
                    # Salary history table might not exist, continue without it
                    pass
                
                # Commit transaction (automatic with context manager)
                conn.commit()
                print(f"    ✓ Transfer completed successfully")
                return True
                
        except sqlite3.Error as e:
            print(f"    ✗ Transfer failed: {e}")
            # Rollback is automatic with context manager on exception
            return False
    
    # Test the transfer function
    success = transfer_employee_to_department(10, 2, 68000)  # Move Jack to Marketing with raise
    
    # 2. Complex transaction with multiple operations
    print("\n2. Complex Multi-Operation Transaction:")
    
    def create_new_project_with_team(project_name: str, dept_id: int, budget: float, 
                                   team_members: List[Tuple[int, str, float]]) -> bool:
        """
        Create a new project and assign team members in a single transaction.
        
        Args:
            project_name: Name of the new project
            dept_id: Department ID
            budget: Project budget
            team_members: List of (employee_id, role, hours_allocated) tuples
        """
        try:
            with get_db_connection() as conn:
                print(f"    Creating project: {project_name}")
                
                # Insert new project
                cursor = conn.execute(
                    "INSERT INTO projects (project_name, start_date, budget, status, department_id) VALUES (?, date('now'), ?, 'PLANNING', ?)",
                    (project_name, budget, dept_id)
                )
                
                project_id = cursor.lastrowid
                print(f"    Project created with ID: {project_id}")
                
                # Assign team members
                total_hours = 0
                for employee_id, role, hours_allocated in team_members:
                    # Verify employee exists and is active
                    cursor = conn.execute(
                        "SELECT first_name, last_name FROM employees WHERE employee_id = ? AND is_active = 1",
                        (employee_id,)
                    )
                    employee = cursor.fetchone()
                    
                    if not employee:
                        raise sqlite3.Error(f"Employee {employee_id} not found or inactive")
                    
                    # Create assignment
                    conn.execute(
                        "INSERT INTO project_assignments (employee_id, project_id, role, hours_allocated, start_date) VALUES (?, ?, ?, ?, date('now'))",
                        (employee_id, project_id, role, hours_allocated)
                    )
                    
                    total_hours += hours_allocated
                    print(f"    Assigned {employee['first_name']} {employee['last_name']} as {role} ({hours_allocated}h/week)")
                
                print(f"    Total team allocation: {total_hours} hours/week")
                print(f"    Budget per hour/week: ${budget/total_hours:.2f}")
                
                # Commit all changes
                conn.commit()
                print(f"    ✓ Project and team assignments created successfully")
                return True
                
        except sqlite3.Error as e:
            print(f"    ✗ Project creation failed: {e}")
            return False
    
    # Test complex transaction
    team_members = [
        (2, "Technical Lead", 40),  # Bob
        (3, "Developer", 35),       # Charlie
        (5, "Marketing Support", 20) # Eve
    ]
    
    success = create_new_project_with_team(
        "Mobile App Development", 
        1,  # Engineering department
        180000, 
        team_members
    )
    
    # 3. Transaction isolation and rollback example
    print("\n3. Transaction Rollback Example:")
    
    def attempt_invalid_operation() -> bool:
        """
        Demonstrate transaction rollback on error.
        """
        try:
            with get_db_connection() as conn:
                print(f"    Starting transaction with multiple operations...")
                
                # Valid operation
                conn.execute(
                    "UPDATE employees SET salary = salary * 1.05 WHERE department_id = 1"
                )
                print(f"    ✓ Applied 5% raise to Engineering department")
                
                # Invalid operation (will cause rollback)
                conn.execute(
                    "INSERT INTO employees (employee_id, first_name, last_name, email, hire_date, department_id, salary) VALUES (1, 'Duplicate', 'Employee', 'duplicate@company.com', '2024-01-01', 1, 50000)"
                )
                print(f"    ✓ Added new employee (this shouldn't appear if rollback works)")
                
                conn.commit()
                return True
                
        except sqlite3.Error as e:
            print(f"    ✗ Transaction failed: {e}")
            print(f"    ↩ All changes rolled back automatically")
            return False
    
    # This should fail due to primary key constraint
    attempt_invalid_operation()
    
    # Verify rollback worked
    try:
        with get_db_connection() as conn:
            cursor = conn.execute("SELECT COUNT(*) as count FROM employees WHERE employee_id = 1")
            count = cursor.fetchone()['count']
            print(f"    Verification: Employee ID 1 exists {count} time(s) (should be 1)")
            
    except sqlite3.Error as e:
        print(f"Error in verification: {e}")

demonstrate_transaction_management()